In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
def crawl_dbp(word):
    url = f'https://prpm.dbp.gov.my/Cari1?keyword={word}'
    r = requests.get(url)
    soup = BeautifulSoup(r._content)
    definitions = soup.select("div[class*=tab-pane]")
    definitions = [d.text for d in definitions]
    spans = soup.find_all('span', id = 'MainContent_SearchInfoTesaurus_lblTesaurus')
    if len(spans):
        tesaurus = spans[0]
        tds = tesaurus.findAll('td')
        selected_td = None
        for td in tds:
            if 'javascript:showModalDialog' in str(td):
                selected_td = td
        if selected_td:
            tesaurus = [a.text for a in selected_td.find_all('a')]
        else:
            tesaurus = None
    else:
        tesaurus = None
    return {'definisi': definitions, 'tesaurus': tesaurus}

In [4]:
import malaya

/home/ubuntu/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [5]:
%%time

crawl_dbp('jaja')

CPU times: user 7.77 ms, sys: 7.84 ms, total: 15.6 ms
Wall time: 371 ms


{'definisi': ['[ja.ja]  |  جاجDefinisi : ; berjaja 1. mencari rezeki (nafkah, pen\xaddapatan, dsb) dgn membawa barang dagangan berkeliling utk dijual: Yahya didenda kerana ~ tanpa lesen; 2. berjual: dia dilarang ~ di situ; menjajakan 1. menjual barang dagangan dgn membawanya berkeliling: riuh-rendah bunyi orang ~ barang dagangan masing-masing; 2. ki menyebarkan (menyatakan dsb) ke sana sini atau merata-rata: Menteri Pen\xaddidikan menasihatkan rakyat supaya tidak terperangkap dgn fikiran sempit go\xadlongan yg ~ pendapat mereka sbg sesuatu yg mutlak; ~ khabar; jajaan benda atau barang yg dijajakan; penjajaan perbuatan (kerja) menjajakan; \xadpenjaja orang yg berjaja: seorang ~ telah di\xadkenakan denda sebanyak RM100.\xa0(Kamus Dewan Edisi Keempat)',
  '[ja.ja]  |  جاجDefinisi : ; berjaja membawa barang-barang dagangan ke merata-rata tempat utk dijual: Tiap-tiap pagi, Jalal ~ nasi lemak. menjajakan menjual sesuatu dgn membawanya berkeliling: Ia pergi dr rumah ke rumah utk ~ hasil kebun

In [6]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [7]:
import mp
import os
import json
from tqdm import tqdm
import traceback
import redis

In [8]:
db = redis.Redis(host='localhost', port=6379, db=14)

In [9]:
len(db.keys())

0

In [24]:
db.keys()

[b'perencat', b'poket', b'basmati', b'perenggannya', b'tohok-menohok']

In [11]:
def loop(words):
    words, _ = words
    db = redis.Redis(host='localhost', port=6379, db=14)
    for word in tqdm(words):
        if word in db:
            continue
        try:
            r = crawl_dbp(word)
            db.set(word, json.dumps(r))
        except Exception as e:
            print(e)
            # print(traceback.format_exc())

In [12]:
with open('malay-words-v5.json') as fopen:
    words = json.load(fopen)
    
len(words)

51655

In [13]:
words[:10]

['basmati',
 'perenggannya',
 'tohok-menohok',
 'poket',
 'perencat',
 'rangkung',
 'panitera',
 'krep',
 'bisulnya',
 'menampek']

In [14]:
stemmer = malaya.stem.deep_model(model = 'noisy')

2022-09-17 15:27:27.336408: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-17 15:27:27.349078: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-17 15:27:27.349269: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-09-17 15:27:27.349274: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-09-17 15:27:27.349324: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-09-17 15:27:27.349586: I

In [22]:
stemmer.stem_word('perenggannya')

'perenggan'

In [16]:
from tqdm import tqdm

stemmed_words = []
for w in tqdm(words):
    stemmed = stemmer.stem_word(w)
    stemmed_words.append(stemmed)

100%|████████████████████████████████████████████████████████████████████████████████████| 51655/51655 [03:58<00:00, 216.53it/s]


In [20]:
len(stemmed_words)

51655

In [21]:
new_words = set(stemmed_words) | set(words)
len(new_words)

54430

In [23]:
loop((words[:5], 0),)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10804.49it/s]


In [29]:
mp.multiprocessing(words, loop, cores = 50, returned = False)

 55%|████████████████████████████████████████████████▎                                       | 567/1033 [31:38<25:26,  3.28s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

